In this notebook we are going to try to assess the performances of USAD, but considering exclusively one decoder, instead of two. The performances will be assessed on the SWAT dataset

In [ ]:
%cd /nfs/home/medoro/Unsupervised_Anomaly_Detection_thesis

In [ ]:
from preprocessing import *
import preprocessing as prp
import pandas as pd
import torch
import torch.nn as nn
import torch.utils.data as data_utils
#from usad import *
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, roc_auc_score
from postprocessing import *
import plotly.graph_objects as go
from USAD.usad_conv import *

import warnings
warnings.filterwarnings('ignore')

# Dataset Preparation

In [ ]:
dataframe = pd.read_csv(r"/nfs/home/medoro/Unsupervised_Anomaly_Detection_thesis/data/train.csv")
dataframe.shape

In [ ]:
df=dataframe[['building_id','primary_use', 'timestamp', 'meter_reading', 'sea_level_pressure', 'is_holiday','anomaly']]
df

In [ ]:
imputed_df = impute_nulls(df)
imputed_df

Now that we have imputed the missing values for the column containing the energy consumption measurements, we can procees by adding a couple of features more and further imputing the missing dates for each timeseries in the dataset.

In [ ]:
df = add_trigonometric_features(imputed_df)
df

In [ ]:
dfs_dict = impute_missing_dates(df)

In [ ]:
dfs_dict

In [ ]:
df1 = pd.concat(dfs_dict.values())
df1

Let's now obtain the train and validation set. We are going to split the dataset into 2 sets, according to the building id.

In [ ]:
dfs_train, dfs_val, dfs_test = train_val_test_split(df1)
train = pd.concat(dfs_train.values())

In [ ]:
val = pd.concat(dfs_val.values())

In [ ]:
test = pd.concat(dfs_test.values())

In [ ]:
train

In [ ]:
val

In [ ]:
test

# Training

In [ ]:
train_window = 72

In [ ]:
X_train, y_train = create_train_eval_sequences(train, train_window)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
BATCH_SIZE =  128
N_EPOCHS = 40
hidden_size = 1/8

In [ ]:
w_size = X_train.shape[1] * X_train.shape[2]
z_size = w_size * hidden_size #X_train.shape[1] * hidden_size 
w_size, z_size

In [ ]:
z_size = int(z_size)

In [ ]:
z_size

In [ ]:
import torch.utils.data as data_utils

In [ ]:
train_loader = torch.utils.data.DataLoader(data_utils.TensorDataset(torch.from_numpy(X_train).float().view(([X_train.shape[0], w_size, 1]))), batch_size = BATCH_SIZE, shuffle = False, num_workers = 0)

In [ ]:
X_val, y_val = create_train_eval_sequences(val, train_window)

In [ ]:
X_val.shape, y_val.shape

In [ ]:
val_loader = torch.utils.data.DataLoader(data_utils.TensorDataset(torch.from_numpy(X_val).float().view(([X_val.shape[0],w_size, 1]))) , batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [ ]:
device = get_default_device()
device

In [ ]:
! export CUDA_VISIBLE_DEVICES=2

In [ ]:
model = UsadModel(w_size, z_size)
model = to_device(model,device)

In [ ]:
N_EPOCHS = 50

In [ ]:
history = training(N_EPOCHS,model,train_loader,val_loader) #2.15 min a epoch ---> 7/11% gpu (uni/multi) #Conv_autoencoder: 2.45 min --> 22% gpu

In [ ]:
plot_history(history)

In [ ]:
print(model)

In [ ]:
torch.save({
            'encoder': model.encoder.state_dict(),
            'decoder1': model.decoder1.state_dict(),
            'decoder2': model.decoder2.state_dict()
            }, "/home/medoro/Unsupervised_Anomaly_Detection_thesis/checkpoints/model_100epochs_univariate.pth")

# Testing the model

In [ ]:
checkpoint = torch.load("checkpoints/model_50epochs_uni_conv.pth")

model.encoder.load_state_dict(checkpoint['encoder'])
model.decoder1.load_state_dict(checkpoint['decoder1'])
model.decoder2.load_state_dict(checkpoint['decoder2'])

In [ ]:
X_test, y_test = create_train_eval_sequences(test, train_window)

In [ ]:
X_test.shape, y_test.shape

## Testing with non-overlapping windows

In [ ]:
X_test, y_test = create_test_sequences(test, train_window) #creo sequenze non overlappate

In [ ]:
X_test.shape, y_test.shape #non-overlapping

In [ ]:
test_loader = torch.utils.data.DataLoader(data_utils.TensorDataset(
    torch.from_numpy(X_test).float().view(([X_test.shape[0],w_size, 1]))
) , batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [ ]:
def reconstruction(model, test_loader):
  # QUI: il test loader che viene passato è ottenuto con non-overlapping sliding window
  tensors_w1 = []
  tensors_w2 = []
  with torch.no_grad():
      for [batch] in test_loader: #N.B.: batch, w1, w2 sono tensori torch.tensor
          batch=to_device(batch,device)
          w1=model.decoder1(model.encoder(batch))
          w2=model.decoder2(model.encoder(w1))
          tensors_w1.append(w1)
          tensors_w2.append(w2)
  # Restituisci solo le ricostruzioni da parte dei due autoencoder
  # Per determinare le anomalie: come facevamo con le baseline, da capire solo come mettere insieme i risultati del primo e del secondo decoder
  # Forse anche qui possiamo calcolare le loss, e almeno per il momento farne una media pesata... no?
  return tensors_w1, tensors_w2

In [ ]:
w1_non_overl, w2_non_overl = reconstruction(model, test_loader)

In [ ]:
w1_non_overl.size()

In [ ]:
w1_non_overl

In [ ]:
# CELLA DA FAR GIRARE SE SI STA USANDO usad_conv
w1_new = [torch.reshape(w1_el, (w1_el.size()[0], w1_el.size()[1])) for w1_el in w1_non_overl]
w2_new = [torch.reshape(w2_el, (w2_el.size()[0], w2_el.size()[1])) for w2_el in w2_non_overl]

In [ ]:
len(w1_non_overl), w1_non_overl[0].size(), w1_non_overl[-1].size()

In [ ]:
36*128 + 28

Now that we have our results, given that the input consisted in non overlapping windows, we can just concatenate the values, into creating a single list of reconstructed values, and then perform anomaly detection as usual, by considering the difference with respect to the ground truth.

In [ ]:
# Operations to do for w1 (output of the first autoencoder)
reshaped_w1 = [torch.flatten(w1_el) for w1_el in w1_non_overl]

In [ ]:
len(reshaped_w1), reshaped_w1[0].size(), reshaped_w1[-1].size()

In [ ]:
# Per i primi 36 tensori, che hanno stessa size, possiamo usare stack per ottenere un unico tensore di 36 * 9216 = 331776 elementi
stacked = torch.stack(reshaped_w1[:-1]).flatten()
stacked.shape

In [ ]:
stacked_array = stacked.cpu().numpy()
stacked_array

In [ ]:
last_array = reshaped_w1[-1].cpu().numpy()
last_array

In [ ]:
total = np.concatenate([stacked_array, last_array])

In [ ]:
len(total)

In [ ]:
# SAME for w2
reshaped_w2 = [torch.flatten(w2_el) for w2_el in w2_non_overl]
stacked2 = torch.stack(reshaped_w2[:-1]).flatten()
stacked_array2 = stacked2.cpu().numpy()
last_array2 = reshaped_w2[-1].cpu().numpy()
total2 = np.concatenate([stacked_array2, last_array2])
len(total2)

In [ ]:
total2

Now we need to create the dataset to perform anomaly detection

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))

In [ ]:
dfs_dict_1 = {}
for building_id, gdf in test.groupby("building_id"):
  gdf[['meter_reading']]=scaler.fit_transform(gdf[['meter_reading']])
  dfs_dict_1[building_id] = gdf
predicted_df_test = pd.concat(dfs_dict_1.values())

In [ ]:
predicted_df_test['reconstruction1'] = total

In [ ]:
predicted_df_test['reconstruction2'] = total2

In [ ]:
predicted_df_test

In [ ]:
predicted_df_test.reconstruction2.min(), predicted_df_test.reconstruction2.max()

In [ ]:
predicted_df_test.reconstruction1.min(), predicted_df_test.reconstruction1.max()

In [ ]:
predicted_df_test['relative_loss'] = np.abs((predicted_df_test['reconstruction1']-predicted_df_test['meter_reading'])/predicted_df_test['reconstruction1'])

In [ ]:
predicted_df_test['relative_loss2'] = np.abs((predicted_df_test['reconstruction2']-predicted_df_test['meter_reading'])/predicted_df_test['reconstruction2'])

In [ ]:
#calculate threshold on relative loss quartiles but only on val, and in this case per building
thresholds=np.array([])
for building_id, gdf in predicted_df_test.groupby("building_id"):
  val_mre_loss_building= gdf['relative_loss'].values
  building_threshold = (np.percentile(val_mre_loss_building, 75)) + 1.5 *((np.percentile(val_mre_loss_building, 75))-(np.percentile(val_mre_loss_building, 25)))
  gdf['threshold']=building_threshold
  thresholds= np.append(thresholds, gdf['threshold'].values)
print(thresholds.shape)
predicted_df_test['threshold']= thresholds

In [ ]:
#calculate threshold on relative loss quartiles but only on val, and in this case per building
thresholds=np.array([])
for building_id, gdf in predicted_df_test.groupby("building_id"):
  val_mre_loss_building= gdf['relative_loss2'].values
  building_threshold = (np.percentile(val_mre_loss_building, 75)) + 1.5 *((np.percentile(val_mre_loss_building, 75))-(np.percentile(val_mre_loss_building, 25)))
  gdf['threshold2']=building_threshold
  thresholds= np.append(thresholds, gdf['threshold2'].values)
print(thresholds.shape)
predicted_df_test['threshold2']= thresholds

In [ ]:
predicted_df_test['predicted_anomaly'] = predicted_df_test['relative_loss'] > predicted_df_test['threshold']
predicted_df_test['predicted_anomaly']=predicted_df_test['predicted_anomaly'].replace(False,0)
predicted_df_test['predicted_anomaly']=predicted_df_test['predicted_anomaly'].replace(True,1)

In [ ]:
predicted_df_test['predicted_anomaly2'] = predicted_df_test['relative_loss2'] > predicted_df_test['threshold2']
predicted_df_test['predicted_anomaly2']=predicted_df_test['predicted_anomaly2'].replace(False,0)
predicted_df_test['predicted_anomaly2']=predicted_df_test['predicted_anomaly2'].replace(True,1)

In [ ]:
predicted_df_test.index.names=['timestamp']
predicted_df_test= predicted_df_test.reset_index()

In [ ]:
predicted_df_test.predicted_anomaly.unique()

In [ ]:
predicted_df_test.predicted_anomaly2.unique()

In [ ]:
predicted_anomalies = predicted_df_test.loc[predicted_df_test['predicted_anomaly'] == 1]
predicted_anomalies2 = predicted_df_test.loc[predicted_df_test['predicted_anomaly2'] == 1]
true_anomalies = predicted_df_test.loc[predicted_df_test['anomaly'] == 1]

In [ ]:
predicted_df_test = pd.merge(predicted_df_test, df[['timestamp','building_id']], on=['timestamp','building_id'])

In [ ]:
print(classification_report(predicted_df_test['anomaly'], predicted_df_test['predicted_anomaly']))

In [ ]:
print(classification_report(predicted_df_test['anomaly'], predicted_df_test['predicted_anomaly2']))

In [ ]:
roc_auc_score(predicted_df_test['anomaly'], predicted_df_test['predicted_anomaly'])

In [ ]:
roc_auc_score(predicted_df_test['anomaly'], predicted_df_test['predicted_anomaly2'])

In [ ]:
len(true_anomalies), len(predicted_anomalies), len(predicted_anomalies2)

In [ ]:
len(predicted_anomalies) / len(predicted_df_test), len(predicted_anomalies2) / len(predicted_df_test)

In [ ]:
predicted_df_test.building_id.unique()

In [ ]:
visualizations = predicted_df_test[predicted_df_test.building_id == 1264]
visualizations

In [ ]:
plt.plot(visualizations.meter_reading, label = "meter reading") 
plt.plot(visualizations.reconstruction, label = "w1_reconstruction")
plt.plot(visualizations.reconstruction2, label = "w2_reconstruction")
plt.legend()
plt.show()

In [ ]:
predicted_anomalies = visualizations.loc[visualizations['predicted_anomaly'] == 1]
predicted_anomalies2 = visualizations.loc[visualizations['predicted_anomaly2'] == 1]
true_anomalies = visualizations.loc[visualizations['anomaly'] == 1]

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=visualizations.index, y=visualizations['meter_reading'], name='meter readings'))
fig.add_trace(go.Scatter(x=visualizations.index, y=visualizations['reconstruction'], name='w1 reconstructed'))
fig.add_trace(go.Scatter(x=visualizations.index, y=visualizations['reconstruction2'], name='w2 reconstructed'))

fig.add_trace(go.Scatter(x=true_anomalies.index, y=true_anomalies['meter_reading'], mode='markers', marker=dict(color='forestgreen'), name='True_Anomaly'))
fig.add_trace(go.Scatter(x=predicted_anomalies.index, y=predicted_anomalies['meter_reading'], mode='markers', marker=dict(color='yellow'), name='W1_Anomaly'))
fig.add_trace(go.Scatter(x=predicted_anomalies2.index, y=predicted_anomalies2['meter_reading'], mode='markers', marker=dict(color='orange'), name='W2_Anomaly'))
fig.update_layout(showlegend=True, title='meter readings predicted and anomalies - val')
fig.show()

# Testing the model (overlapping windows; anomaly score-based)

In [ ]:
X_test, y_test = create_train_eval_sequences(test, train_window)

In [ ]:
X_test.shape, y_test.shape

In [ ]:
test_loader = torch.utils.data.DataLoader(data_utils.TensorDataset(
    torch.from_numpy(X_test).float().view(([X_test.shape[0],w_size, 1]))
) , batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [ ]:
results=testing(model,test_loader) #Prova con il test set

In [ ]:
results

Let's create the dataset to perform predictions.

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))

In [ ]:
dfs_dict_1 = {}
for building_id, gdf in test.groupby("building_id"):
  gdf[['meter_reading', 'sea_level_pressure']]=scaler.fit_transform(gdf[['meter_reading', 'sea_level_pressure']])
  dfs_dict_1[building_id] = gdf[train_window:]
predicted_df = pd.concat(dfs_dict_1.values())

In [ ]:
lista = []
for el in results:
  for el2 in el:
    lista.append(el2.cpu().item())
lista

In [ ]:
plt.hist(lista, bins=50)
plt.xlabel("Anomaly score")
plt.ylabel("No of samples")
plt.show()

In [ ]:
predicted_df['anomaly_score'] = lista

In [ ]:
predicted_df

In [ ]:
predicted_df.anomaly_score.min(), predicted_df.anomaly_score.max() 

In [ ]:
perc = 90
threshold = (np.percentile(predicted_df.anomaly_score.values, perc))

In [ ]:
threshold

In [ ]:
predicted_df['threshold'] = threshold

In [ ]:
predicted_df['predicted_anomaly'] = predicted_df.anomaly_score > predicted_df['threshold']
predicted_df['predicted_anomaly']=predicted_df['predicted_anomaly'].replace(False,0)
predicted_df['predicted_anomaly']=predicted_df['predicted_anomaly'].replace(True,1)

In [ ]:
predicted_df.predicted_anomaly.unique()

In [ ]:
len(predicted_df[predicted_df.predicted_anomaly == 1])/len(predicted_df)

In [ ]:
predicted_df.index.names=['timestamp']
predicted_df= predicted_df.reset_index()

In [ ]:
predicted_df = pd.merge(predicted_df, df[['timestamp','building_id']], on=['timestamp','building_id'])

In [ ]:
print(classification_report(predicted_df.anomaly, predicted_df.predicted_anomaly))

In [ ]:
roc_auc_score(predicted_df['anomaly'], predicted_df['predicted_anomaly'])

# Testing the model (overlapping windows; reconstruction-based)

In [ ]:
results, w1, w2 = testing_prova(model, test_loader)

In [ ]:
w1

In [ ]:
w2

In [ ]:
# CELLA DA FAR GIRARE SE SI STA USANDO usad_conv
w1_new = [torch.reshape(w1_el, (w1_el.size()[0], w1_el.size()[1])) for w1_el in w1]
w2_new = [torch.reshape(w2_el, (w2_el.size()[0], w2_el.size()[1])) for w2_el in w2]

In [ ]:
w1

In [ ]:
padded_w1 = padding_w(w1_new, BATCH_SIZE) # Se si usa usad, non convoluzionale, mettere w1 al posto di w1_new

In [ ]:
padded_w2 = padding_w(w2_new, BATCH_SIZE) # Se si usa usad, non convoluzionale, mettere w2 al posto di w2_new

In [ ]:
reconstruction1 = apply_reconstruction(padded_w1, test.building_id.nunique())

In [ ]:
reconstruction2 = apply_reconstruction(padded_w2, test.building_id.nunique())

## Reconstruction (overlapping; w1,w2-based)

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))

In [ ]:
dfs_dict_1 = {}
for building_id, gdf in test.groupby("building_id"):
  gdf[['meter_reading']]=scaler.fit_transform(gdf[['meter_reading']])
  dfs_dict_1[building_id] = gdf
predicted_df_test = pd.concat(dfs_dict_1.values())

In [ ]:
predicted_df_test['reconstruction'] = reconstruction1

In [ ]:
predicted_df_test['reconstruction2'] = reconstruction2

In [ ]:
predicted_df_test

In [ ]:
predicted_df_test['reconstruction']=predicted_df_test['reconstruction'].replace(np.nan,0)

In [ ]:
predicted_df_test['reconstruction2']=predicted_df_test['reconstruction2'].replace(np.nan,0)

In [ ]:
predicted_df_test

In [ ]:
predicted_df_test.reconstruction2.min(), predicted_df_test.reconstruction2.max()

In [ ]:
predicted_df_test['relative_loss'] = np.abs((predicted_df_test['reconstruction']-predicted_df_test['meter_reading'])/predicted_df_test['reconstruction'])

In [ ]:
predicted_df_test['relative_loss2'] = np.abs((predicted_df_test['reconstruction2']-predicted_df_test['meter_reading'])/predicted_df_test['reconstruction2'])

In [ ]:
#calculate threshold on relative loss quartiles but only on val, and in this case per building
thresholds=np.array([])
for building_id, gdf in predicted_df_test.groupby("building_id"):
  val_mre_loss_building= gdf['relative_loss'].values
  building_threshold = (np.percentile(val_mre_loss_building, 75)) + 1.5 *((np.percentile(val_mre_loss_building, 75))-(np.percentile(val_mre_loss_building, 25)))
  gdf['threshold']=building_threshold
  thresholds= np.append(thresholds, gdf['threshold'].values)
print(thresholds.shape)
predicted_df_test['threshold']= thresholds

In [ ]:
#calculate threshold on relative loss quartiles but only on val, and in this case per building
thresholds=np.array([])
for building_id, gdf in predicted_df_test.groupby("building_id"):
  val_mre_loss_building= gdf['relative_loss2'].values
  building_threshold = (np.percentile(val_mre_loss_building, 75)) + 1.5 *((np.percentile(val_mre_loss_building, 75))-(np.percentile(val_mre_loss_building, 25)))
  gdf['threshold2']=building_threshold
  thresholds= np.append(thresholds, gdf['threshold2'].values)
print(thresholds.shape)
predicted_df_test['threshold2']= thresholds

In [ ]:
predicted_df_test

In [ ]:
predicted_df_test['predicted_anomaly'] = predicted_df_test['relative_loss'] > predicted_df_test['threshold']
predicted_df_test['predicted_anomaly']=predicted_df_test['predicted_anomaly'].replace(False,0)
predicted_df_test['predicted_anomaly']=predicted_df_test['predicted_anomaly'].replace(True,1)

In [ ]:
predicted_df_test['predicted_anomaly2'] = predicted_df_test['relative_loss2'] > predicted_df_test['threshold2']
predicted_df_test['predicted_anomaly2']=predicted_df_test['predicted_anomaly2'].replace(False,0)
predicted_df_test['predicted_anomaly2']=predicted_df_test['predicted_anomaly2'].replace(True,1)

In [ ]:
predicted_df_test.index.names=['timestamp']
predicted_df_test= predicted_df_test.reset_index()

In [ ]:
predicted_df_test.predicted_anomaly.unique()

In [ ]:
predicted_df_test.predicted_anomaly2.unique()

In [ ]:
predicted_df_test.predicted_anomaly.sum() / len(predicted_df_test), predicted_df_test.predicted_anomaly2.sum() / len(predicted_df_test)

In [ ]:
predicted_anomalies = predicted_df_test.loc[predicted_df_test['predicted_anomaly'] == 1]
predicted_anomalies2 = predicted_df_test.loc[predicted_df_test['predicted_anomaly2'] == 1]
true_anomalies = predicted_df_test.loc[predicted_df_test['anomaly'] == 1]

In [ ]:
len(predicted_anomalies) / len(predicted_df_test), len(predicted_anomalies2) / len(predicted_df_test)

In [ ]:
predicted_df_test = pd.merge(predicted_df_test, df[['timestamp','building_id']], on=['timestamp','building_id'])

In [ ]:
print(classification_report(predicted_df_test['anomaly'], predicted_df_test['predicted_anomaly']))

In [ ]:
print(classification_report(predicted_df_test['anomaly'], predicted_df_test['predicted_anomaly2']))

In [ ]:
roc_auc_score(predicted_df_test['anomaly'], predicted_df_test['predicted_anomaly'])

In [ ]:
roc_auc_score(predicted_df_test['anomaly'], predicted_df_test['predicted_anomaly2'])

In [ ]:
predicted_df_test.building_id.unique()

In [ ]:
visualizations = predicted_df_test[predicted_df_test.building_id == 994]
visualizations

In [ ]:
plt.plot(visualizations.meter_reading, label = "meter reading") #predicted_df_test.meter_reading[:8784]
plt.plot(visualizations.reconstruction, label = "w1_reconstruction")
plt.plot(visualizations.reconstruction2, label = "w2_reconstruction")
plt.legend()
plt.show()

In [ ]:
predicted_anomalies = visualizations.loc[visualizations['predicted_anomaly'] == 1]
predicted_anomalies2 = visualizations.loc[visualizations['predicted_anomaly2'] == 1]
true_anomalies = visualizations.loc[visualizations['anomaly'] == 1]

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=visualizations.index, y=visualizations['meter_reading'], name='meter readings'))
fig.add_trace(go.Scatter(x=visualizations.index, y=visualizations['reconstruction'], name='w1 reconstructed'))
fig.add_trace(go.Scatter(x=visualizations.index, y=visualizations['reconstruction2'], name='w2 reconstructed'))

fig.add_trace(go.Scatter(x=true_anomalies.index, y=true_anomalies['meter_reading'], mode='markers', marker=dict(color='forestgreen'), name='True_Anomaly'))
fig.add_trace(go.Scatter(x=predicted_anomalies.index, y=predicted_anomalies['meter_reading'], mode='markers', marker=dict(color='yellow'), name='W1_Anomaly'))
fig.add_trace(go.Scatter(x=predicted_anomalies2.index, y=predicted_anomalies2['meter_reading'], mode='markers', marker=dict(color='orange'), name='W2_Anomaly'))
fig.update_layout(showlegend=True, title='meter readings predicted and anomalies - val')
fig.show()

In [ ]:
len(true_anomalies), len(predicted_anomalies), len(predicted_anomalies2)